# Baseball Play Description Inference Model - Test Response Creator

This notebook runs code that generates and saves responses from a trained LLM for a set test prompts.

## Step 0: Load Necessary Python Modules

The module **peft** is not included in the Google Colab environment, but it are required to load the trained model and generate responses. Upon opening this notebook or connecting to a new runtime, run this cell and restart the runtime to let the changes take effect.

In [ ]:
!pip install peft

## Step 1: Mount Google Drive and Import Functions

The following cell connects the notebook to Google Drive, imports the necessary functions from outside models, and sets some options.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer
from peft import PeftConfig, PeftModel, AutoPeftModelForSeq2SeqLM
from pathlib import Path
import pandas as pd
filepath = Path('/content/drive/MyDrive/Text Mining Project')

Mounted at /content/drive


## Step 2: Load and Prepare the Model

The following cell loads a previously fine-tuned language model, a tokenizer, and a dataset with test prompts that will be used to generate responses.

In [ ]:
model_name = filepath / 'baseball-flan-t5'
model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)
test_df = pd.read_csv(filepath / 'test_game_docs.csv').iloc[:,1:]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Step 3: Generate Responses

The following cell generates responses from the test prompts in the dataset. Uncommenting the commented lines allows you to print the responses alongside the target response for easy comparison.

In [ ]:
prompts = test_df['document']
scores = test_df['score']
test_responses = []

# i = 0
for prompt in prompts:
  input = tokenizer(prompt, max_length = 810, truncation = True,
                    return_tensors = 'pt').to('cuda')
  output = tokenizer.decode(
    model.generate(
        input_ids = input['input_ids'],
        max_new_tokens = 25,
        do_sample = True
    )[0],
    skip_special_tokens = True
    )
  # print(f'True Score: {scores[i]}')
  # print(f'Response: {output}\n')
  # i += 1
  test_responses.append(output)

## Step 4: Export the Responses for Analysis

In [ ]:
test_df.insert(2, "response", test_responses)
test_df.to_csv(filepath / 'game_responses.csv')